In [59]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from datetime import time
import os



In [60]:
raw_file="Raw data 26-April'25 According to 4_7.5.csv"
gst_file="GST 26-April'25 According to 4_7.5.csv"
tracker_file="EDD Tracker till 26th April'25.csv"
format_file="edd_format.csv"
ops_file="ops_status.csv"
old_tracker_file="old_tracker.csv"


In [61]:
raw_data= pd.read_csv(raw_file,memory_map=True)
gst_data=pd.read_csv(gst_file,memory_map=True)
tracker_data=pd.read_csv(tracker_file,memory_map=True)
master_format=pd.read_csv(format_file,memory_map=True)
ops_status=pd.read_csv(ops_file,memory_map=True)
old_tracker=pd.read_csv(old_tracker_file,memory_map=True)

In [62]:
raw_data.head(3)

,parent_entity_id,parent_id,mid,entity_id,mid_creation_date,channel_date,cust_id,channel,edc_flag,vpa_type,...,gmv,limit_flag,drr,month_expected_gmv,total_child_mid,rank,limit_usage,_required_flag,pending_at_,propose_limit
0,84152466,wGkLWY00056042674942,wGkLWY00056042674942,84152466,2024-06-11 13:33:56.000,2024-06-11 13:39:34.000,1.899359e+09,DIY,QR,PTY,...,46255.0,400000,1595.0,49445.0,1,1,0.115638,Not_required,Pending for share,100000.0
1,87831302,wIoyDM46851761551383,wIoyDM46851761551383,87831302,2025-04-21 16:16:44.000,2025-04-21 16:17:17.000,7.043959e+06,Transit Offline,EDC,PTY,...,NaN,750000,NaN,NaN,1,1,NaN,Not_required,Pending for share,NaN
2,85538028,vmiSgy43648835767287,vmiSgy43648835767287,85538028,2024-10-26 12:57:41.000,2024-10-26 12:58:49.000,1.745045e+09,Offline Unorganized,EDC,PTY,...,NaN,750000,NaN,NaN,1,1,NaN,Not_required,Pending for share,NaN


In [63]:
gst_data=gst_data.rename(columns={'_col3':'updated_at'})
gst_data.head(3)

,solution_value,cust_id,solution_value.1,updated_at
0,mCJLRw40248294050664,1.950030e+09,Slab: Rs. 40 lakhs to 1.5 Cr.,2025-04-26
1,EcuCam75972053723310,1.278551e+09,Slab: Rs. 0 to 40 lakhs,2025-04-26
2,nRccGs29005105680621,5.203776e+07,Slab: Rs. 0 to 40 lakhs,2025-04-25


In [64]:
# Convert the two columns to datetime first
raw_data['mid_creation_date'] = pd.to_datetime(raw_data['mid_creation_date'], errors='coerce')
raw_data['channel_date'] = pd.to_datetime(raw_data['channel_date'], errors='coerce')

# Then format them to 'dd-mm-yyyy' string format
raw_data['mid_creation_date'] = raw_data['mid_creation_date'].dt.strftime('%d-%m-%Y')
raw_data['channel_date'] = raw_data['channel_date'].dt.strftime('%d-%m-%Y')

raw_data.head(3)

,parent_entity_id,parent_id,mid,entity_id,mid_creation_date,channel_date,cust_id,channel,edc_flag,vpa_type,...,gmv,limit_flag,drr,month_expected_gmv,total_child_mid,rank,limit_usage,_required_flag,pending_at_,propose_limit
0,84152466,wGkLWY00056042674942,wGkLWY00056042674942,84152466,11-06-2024,11-06-2024,1.899359e+09,DIY,QR,PTY,...,46255.0,400000,1595.0,49445.0,1,1,0.115638,Not_required,Pending for share,100000.0
1,87831302,wIoyDM46851761551383,wIoyDM46851761551383,87831302,21-04-2025,21-04-2025,7.043959e+06,Transit Offline,EDC,PTY,...,NaN,750000,NaN,NaN,1,1,NaN,Not_required,Pending for share,NaN
2,85538028,vmiSgy43648835767287,vmiSgy43648835767287,85538028,26-10-2024,26-10-2024,1.745045e+09,Offline Unorganized,EDC,PTY,...,NaN,750000,NaN,NaN,1,1,NaN,Not_required,Pending for share,NaN


In [65]:
tracker_data=tracker_data.fillna(0)
tracker_data.isna().sum()

Bank Response    0
Final Remarks    0
dtype: int64

In [66]:
len(ops_status)+len(tracker_data)+len(old_tracker)

316948

In [67]:
tracker_data=tracker_data.rename(columns={'Bank Response':'bank_response','Final Remarks':'final_remarks'})
old_tracker=old_tracker.rename(columns={'bank_response':'bank_response','Final Remarks':'final_remarks'})
ops_status=ops_status.rename(columns={'sellerid':'bank_response','OPS status':'final_remarks'})



ops_status_data= pd.concat([ops_status, tracker_data,old_tracker], axis=0, ignore_index=True)
print('totol tracker data ',len(ops_status_data))
print(ops_status_data.isna().sum())


totol tracker data  316948
bank_response    0
final_remarks    0
dtype: int64
